In [ ]:
"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\hlsp_deepmerge_hst_acs-wfc3_illustris-z2_f814w-f160w_v1_sim-pristine.fits"
r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf.txt", dtype=None, encoding='utf-8'

In [20]:
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import h5py

# Definir os caminhos dos arquivos
file_x = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimSim_SOURCE_X_Illustris2_pristine.npy"
file_y = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimSim_SOURCE_y_Illustris2_pristine.npy"

# Carregar os arquivos usando np.load
data_x = np.load(file_x)
data_y = np.load(file_y)

# Diretório para as imagens
image_dir = "pristineSS"
os.makedirs(image_dir, exist_ok=True)

# Criar um dicionário para os metadados
metadata = {}

# Usar tqdm para a barra de progresso
for i in tqdm(range(data_x.shape[0]), desc="Salvando imagens e metadados"):
    image = data_x[i]
    label = data_y

    # Normaliza os dados da imagem para o intervalo [0, 1]
    if np.max(image) != np.min(image):
        image = (image - np.min(image)) / (np.max(image) - np.min(image))
    else:
        # Handle the case where max and min are equal
        image = np.zeros_like(image)

    # Verifique se os dados são em escala de cinza e converta para RGB
    if image.shape[0] == 1:
        image = plt.cm.viridis(image)

    # Reorganize a forma da imagem para (altura, largura, canais)
    image = np.transpose(image, (1, 2, 0))

    # Salva a imagem como um arquivo PNG
    filename = f"imagem_{i}.png"
    filepath = os.path.join(image_dir, filename)
    plt.imsave(filepath, image)

    # Adiciona a imagem e o rótulo ao dicionário de metadados
    metadata[f"imagem_{i}"] = {"rótulo": label, "arquivo": filename}

# Save the data to an HDF5 file
with h5py.File("pristineSS.hdf5", "w") as f:
    f.create_dataset("data_x", data=data_x)
    f.create_dataset("data_y", data=data_y)

Salvando imagens e metadados: 100%|█| 15426/15426 [00:21<00:00, 7


In [23]:
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import h5py

# Definir os caminhos dos arquivos
file_x = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimReal_SOURCE_X_Illustris0.npy"
file_y = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimReal_SOURCE_y_Illustris0.npy"

# Carregar os arquivos usando np.load
data_x = np.load(file_x)
data_y = np.load(file_y)

# Diretório para as imagens
image_dir = "pristineSR"
os.makedirs(image_dir, exist_ok=True)

# Criar um dicionário para os metadados
metadata = {}

# Usar tqdm para a barra de progresso
for i in tqdm(range(data_x.shape[0]), desc="Salvando imagens e metadados"):
    image = data_x[i]
    label = data_y[i]

    # Normaliza os dados da imagem para o intervalo [0, 1]
    image = (image - np.min(image)) / (np.max(image) - np.min(image))

    # Verifique se os dados são em escala de cinza e converta para RGB
    if image.shape[0] == 1:
        image = plt.cm.viridis(image)

    # Reorganize a forma da imagem para (altura, largura, canais)
    image = np.transpose(image, (1, 2, 0))

    # Salva a imagem como um arquivo PNG
    filename = f"imagem_{i}.png"
    filepath = os.path.join(image_dir, filename)
    plt.imsave(filepath, image)

    # Adiciona a imagem e o rótulo ao dicionário de metadados
    metadata[f"imagem_{i}"] = {"rótulo": label, "arquivo": filename}

# Salva os metadados como um arquivo HDF5
with h5py.File("pristineSR.hdf5", "w") as f:
    f.create_dataset("data_x", data=data_x)
    f.create_dataset("data_y", data=data_y)

Salvando imagens e metadados: 100%|█| 6000/6000 [00:20<00:00, 287


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Caminhos dos arquivos
file_x_simsim = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimSim_SOURCE_X_Illustris2_pristine.npy"
file_y_simsim = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimSim_SOURCE_y_Illustris2_pristine.npy"
file_x_simreal = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimReal_SOURCE_X_Illustris0.npy"
file_y_simreal = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\SimReal_SOURCE_y_Illustris0.npy"

# Carregar os dados
X_simsim = np.load(file_x_simsim)
y_simsim = np.load(file_y_simsim)
X_simreal = np.load(file_x_simreal)
y_simreal = np.load(file_y_simreal)

In [5]:
# Criar um objeto ImageDataGenerator com as transformações desejadas
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=45,
    zoom_range=0.2,
    fill_mode='nearest'
)

In [6]:
# Aumentar os dados de SimSim em batches
X_simsim_augmented = []
y_simsim_augmented = []
batch_size = 32
for i in range(len(X_simsim) // batch_size):
    batch_x = X_simsim[i * batch_size:(i + 1) * batch_size]
    batch_y = y_simsim[i * batch_size:(i + 1) * batch_size]
    
    # Directly iterate through the NumpyArrayIterator
    for batch_augmented in datagen.flow(batch_x, batch_y, batch_size=batch_size, shuffle=False):
        X_simsim_augmented.extend(batch_augmented[0])
        y_simsim_augmented.extend(batch_augmented[1])
        break # Get only one batch per iteration

# Aumentar os dados de SimReal em batches
X_simreal_augmented = []
y_simreal_augmented = []
for i in range(len(X_simreal) // batch_size):
    batch_x = X_simreal[i * batch_size:(i + 1) * batch_size]
    batch_y = y_simreal[i * batch_size:(i + 1) * batch_size]
    
    # Directly iterate through the NumpyArrayIterator
    for batch_augmented in datagen.flow(batch_x, batch_y, batch_size=batch_size, shuffle=False):
        X_simreal_augmented.extend(batch_augmented[0])
        y_simreal_augmented.extend(batch_augmented[1])
        break # Get only one batch per iteration

C:\Users\stefa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\legacy\preprocessing\image.py:619: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (32, 3, 75, 75) (75 channels).
  warnings.warn(


In [7]:
# Convert the augmented lists to NumPy arrays
X_simsim_augmented = np.array(X_simsim_augmented)
y_simsim_augmented = np.array(y_simsim_augmented)
X_simreal_augmented = np.array(X_simreal_augmented)
y_simreal_augmented = np.array(y_simreal_augmented)

In [8]:
# Concatene os dados originais e aumentados
X_simsim_augmented = np.concatenate((X_simsim, np.array(X_simsim_augmented)))
y_simsim_augmented = np.concatenate((y_simsim, y_simsim_augmented))

X_simreal_augmented = np.concatenate((X_simreal, np.array(X_simreal_augmented)))
y_simreal_augmented = np.concatenate((y_simreal, y_simreal_augmented))

In [ ]:
# Salve os dados aumentados
np.save('SimSim_SOURCE_X_Illustris2_pristine_augmented.npy', X_simsim_augmented)
np.save('SimSim_SOURCE_y_Illustris2_pristine_augmented.npy', y_simsim_augmented)

np.save('SimReal_SOURCE_X_Illustris0_augmented.npy', X_simreal_augmented)
np.save('SimReal_SOURCE_y_Illustris0_augmented.npy', y_simreal_augmented)

In [ ]:
# Imprimir o tamanho dos conjuntos de dados aumentados
print(f"SimSim augmented data size: {len(X_simsim_augmented)}")
print(f"SimReal augmented data size: {len(X_simreal_augmented)}")

In [46]:
import numpy as np
import pandas as pd
import glob
import os

# Importing Illustris catalog
catalog = pd.DataFrame(np.genfromtxt(r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf.txt", dtype=None, encoding='utf-8'))
catalog.columns = ['snap', 'subid', 'logmstar', 'cam', 'merger', 'rfprob', 'asym_I', 'cc_I', 'g_I', 'm20_I', 'mprime_I', 'asym_H', 'cc_H', 'g_H', 'm20_H', 'mprime_H', 't_lastmaj', 't_lastmin', 't_nextmaj', 't_nextmin']

print(catalog)

       snap  subid  logmstar  cam  merger rfprob  asym_I    cc_I     g_I  \
0       103      0     12.20    0   False  0.103  0.2076  2.9771  0.5534   
1       103      0     12.20    3   False   0.15  0.2082  3.6671  0.5881   
2       103      0     12.20    1   False  0.132  0.2069  3.6939  0.6117   
3       103      0     12.20    2   False  0.231  0.2195  3.2588  0.5904   
4       103  19702     12.12    3   False   None     NaN     NaN     NaN   
...     ...    ...       ...  ...     ...    ...     ...     ...     ...   
71946    54  18603      9.83    2   False   None -0.0648  2.6204  0.4758   
71947    54   8205      9.82    1   False   None  0.0134  2.4736  0.4804   
71948    54   8205      9.82    3   False   None  0.0519  2.4502  0.4625   
71949    54   8205      9.82    2   False   None  0.1353  2.1423  0.4737   
71950    54   8205      9.82    0   False   None  0.1309  2.4773  0.4857   

        m20_I  mprime_I  asym_H    cc_H     g_H   m20_H  mprime_H  t_lastmaj  \
0     -

In [47]:
# Extract only objects from snapshot 68 (z=2)
merged_68 = catalog[catalog['snap'] == 68]
print(catalog)

       snap  subid  logmstar  cam  merger rfprob  asym_I    cc_I     g_I  \
0       103      0     12.20    0   False  0.103  0.2076  2.9771  0.5534   
1       103      0     12.20    3   False   0.15  0.2082  3.6671  0.5881   
2       103      0     12.20    1   False  0.132  0.2069  3.6939  0.6117   
3       103      0     12.20    2   False  0.231  0.2195  3.2588  0.5904   
4       103  19702     12.12    3   False   None     NaN     NaN     NaN   
...     ...    ...       ...  ...     ...    ...     ...     ...     ...   
71946    54  18603      9.83    2   False   None -0.0648  2.6204  0.4758   
71947    54   8205      9.82    1   False   None  0.0134  2.4736  0.4804   
71948    54   8205      9.82    3   False   None  0.0519  2.4502  0.4625   
71949    54   8205      9.82    2   False   None  0.1353  2.1423  0.4737   
71950    54   8205      9.82    0   False   None  0.1309  2.4773  0.4857   

        m20_I  mprime_I  asym_H    cc_H     g_H   m20_H  mprime_H  t_lastmaj  \
0     -

In [5]:
import pandas as pd
import numpy as np

# Carrega o arquivo "illustris_morphs_rf_snap68.txt"
data = pd.read_csv(r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf_snap68.txt", sep=" ", header=None)

# Assumindo que as colunas do arquivo são: subid, merger_label (e talvez outras)
subsamples_y = data.values[:, [0, 1]]  # Extrai as colunas subid e merger_label

# Salva o array em "subsamples_y.npy"
np.save("subsamples_y.npy", subsamples_y)

print("Arquivo 'subsamples_y.npy' criado com sucesso!")

Arquivo 'subsamples_y.npy' criado com sucesso!


In [33]:
import os
from astropy.io import fits

source_path = r"C:\Users\stefa\OneDrive\Documentos\NATALI TCC\hlsp_deepmerge_hst_acs-wfc3_illustris-z2_f814w-f160w_v1_sim-pristine.fits"

# Obter o nome do arquivo
filename = os.path.basename(source_path)

# Carregar o arquivo FITS
hdulist = fits.open(source_path)

# Obtenha as informações do cabeçalho
header = hdulist[0].header

# Imprima as informações desejadas
print("Nome do arquivo:", filename)  # Use o nome do arquivo
print("Data da observação:", header['DATE-OBS'])
print("Tamanho da imagem:", hdulist[0].data.shape)

# Feche o arquivo FITS
hdulist.close()

Nome do arquivo: hlsp_deepmerge_hst_acs-wfc3_illustris-z2_f814w-f160w_v1_sim-pristine.fits


KeyError: "Keyword 'DATE-OBS' not found."

In [11]:
import h5py
import numpy as np
import pandas as pd

# Caminhos dos arquivos
target_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\extraidos.hdf5"
txt_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf_snap68.txt"

# Carrega o arquivo de texto com pandas
merged_68 = pd.read_csv(txt_path, sep=" ", header=None, 
                      names=['snap', 'subid', 'logmstar', 'cam', 'merger', 
                             'rfprob', 'asym_I', 'cc_I', 'g_I', 'm20_I', 'mprime_I', 
                             'asym_H', 'cc_H', 'g_H', 'm20_H', 'mprime_H', 
                             't_lastmaj', 't_lastmin', 't_nextmaj', 't_nextmin'])

# Cria um array com objetos usados no RF e aqueles excluídos
rf_objects = np.array([[row['subid'], row['merger']] for _, row in merged_68.iterrows()], dtype=None)

# Obtém IDs de objeto exclusivos e seus rótulos de fusão correspondentes
subid, indices = np.unique(rf_objects[:, 0], return_index=True)
RF_labels = rf_objects[indices]

shid_array = RF_labels[:, 0]
merger_labels = RF_labels[:, 1]

# Abre o arquivo HDF5
with h5py.File(target_path, 'r') as f:
    # Itera sobre os grupos
    for group_name in f.keys():
        group = f[group_name]
        print(f"\nGrupo: {group_name}")

        # Itera sobre os datasets dentro do grupo
        for dataset_name in group.keys():
            dataset = group[dataset_name]
            print(f"  Dataset: {dataset_name}")

            # Verifica se o dataset é um filtro F814W ou F160W
            if "F814W" in dataset_name or "F160W" in dataset_name:
                print(f"    Este dataset é um filtro F814W ou F160W")

                # Itera sobre os IDs de subsample exclusivos
                for i, shid in enumerate(shid_array):
                    # Verifica se o ID está presente no dataset
                    if shid in dataset:
                        # Obtém o rótulo de fusão correspondente
                        merger_label = merger_labels[i]
                        print(f"      subid: {shid}, rótulo de fusão: {merger_label}")
                        


Grupo: imagem_0
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_1
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_10
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_11
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_12
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_13
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_14
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_15
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_2
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_3
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_4
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_5
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_6
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_7
  Dataset: f160w
  Dataset: f814w
  Dataset: merger

Grupo: imagem_8
  Dataset: 

In [30]:
import h5py
import numpy as np
import os

# Caminhos dos arquivos
target_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\extraidos.hdf5"

# Cria a lista de subdiretórios
subdirs = ['res_subdir_{:03d}'.format(x) for x in range(23)]

# Abre o arquivo HDF5
with h5py.File(target_path, 'r') as f:
    # Itera sobre os grupos
    for i, group_name in enumerate(f.keys()):
        group = f[group_name]
        print(f"\nGrupo: {group_name}")

        # Itera sobre os subdiretórios
        for subdir in subdirs:
            # Salva os datasets 'f160w' e 'f814w'
            for dataset_name in ['f160w', 'f814w']:
                file_path = os.path.join("C:\\Users\\stefa\\OneDrive\\Documentos\\Natali 2\\", subdir, f"{i}_{dataset_name}.npy")
                print(f"  Salvando arquivo: {file_path}")  # Adiciona um print para mostrar o caminho do arquivo

                # Salva o dataset como um arquivo .npy no subdiretório
                np.save(file_path, group[dataset_name][:])


Grupo: imagem_0
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_000\0_f160w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_000\0_f814w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_001\0_f160w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_001\0_f814w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_002\0_f160w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_002\0_f814w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_003\0_f160w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_003\0_f814w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_004\0_f160w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir_004\0_f814w.npy
  Salvando arquivo: C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdi

In [32]:
import h5py

target_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\extraidos.hdf5"

with h5py.File(target_path, 'r') as f:
    for group_name in f.keys():
        group = f[group_name]
        print(f"\nGrupo: {group_name}")

        for dataset_name in group.keys():
            dataset = group[dataset_name]
            print(f"  Dataset: {dataset_name}")

            # Imprime o formato e o tipo de dados do dataset
            print(f"    Formato: {dataset.shape}")
            print(f"    Tipo de dados: {dataset.dtype}")


Grupo: imagem_0
  Dataset: f160w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: f814w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: merger
    Formato: ()
    Tipo de dados: bool

Grupo: imagem_1
  Dataset: f160w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: f814w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: merger
    Formato: ()
    Tipo de dados: bool

Grupo: imagem_10
  Dataset: f160w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: f814w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: merger
    Formato: ()
    Tipo de dados: bool

Grupo: imagem_11
  Dataset: f160w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: f814w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: merger
    Formato: ()
    Tipo de dados: bool

Grupo: imagem_12
  Dataset: f160w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: f814w
    Formato: (75, 75)
    Tipo de dados: float64
  Dataset: merger
  

In [43]:
import numpy as np
import pandas as pd

# Caminho do arquivo .npy
npy_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf_snap68.npy"

# Carrega os dados do arquivo .npy
data = np.load(npy_path, allow_pickle=True)  # Permite o carregamento de dados pickle

# Carrega o arquivo de texto (opcional)
if txt_path:
    merged_68 = pd.read_csv(txt_path, sep=" ", header=None, 
                          names=['snap', 'subid', 'logmstar', 'cam', 'merger', 
                                 'rfprob', 'asym_I', 'cc_I', 'g_I', 'm20_I', 'mprime_I', 
                                 'asym_H', 'cc_H', 'g_H', 'm20_H', 'mprime_H', 
                                 't_lastmaj', 't_lastmin', 't_nextmaj', 't_nextmin'])

    # Cria um array com objetos usados no RF e aqueles excluídos
    rf_objects = np.array([[row['subid'], row['merger']] for _, row in merged_68.iterrows()], dtype=None)

    # Obtém IDs de objeto exclusivos e seus rótulos de fusão correspondentes
    subid, indices = np.unique(rf_objects[:, 0], return_index=True)
    RF_labels = rf_objects[indices]

    shid_array = RF_labels[:, 0]
    merger_labels = RF_labels[:, 1]

# Separa os dados em fusões e não-fusões (se você tiver os IDs e rótulos)
if txt_path:
    list_of_mergers = [data[i] for i in range(len(data)) if merger_labels[i] == 1]
    list_of_nonmergers = [data[i] for i in range(len(data)) if merger_labels[i] == 0]

# Imprime o formato dos dados dentro de cada lista
print("\nFormato dos dados na lista de fusões:")
if txt_path:
    print(np.array(list_of_mergers).shape)
else:
    print("Lista de fusões está vazia.")

print("\nFormato dos dados na lista de não-fusões:")
if txt_path:
    print(np.array(list_of_nonmergers).shape)
else:
    print("Lista de não-fusões está vazia.")

IndexError: index 2234 is out of bounds for axis 0 with size 2234

In [33]:
# Iterate through objects and their merger labels
for shid, merger in zip(shid_array, merger_labels):
    for camnum in ['00', '01', '02', '03']:
        filter_files = np.sort(np.asarray(glob.glob(os.path.join(path, 'res_sub*', '*sh' + str(shid) + 'cam' + camnum + '*SB00.npy'))))

        # Check if there are exactly 4 files for each object
        if len(filter_files) != 4:
            continue

        # Load image data for all 4 filters
        tmp = np.zeros((4, 75, 75))
        for i, file in enumerate(filter_files):
            tmp[i, :, :] = np.load(file)

        # Append image data to corresponding list based on merger label
        if merger:
            list_of_mergers.append(tmp)
        else:
            list_of_nonmergers.append(tmp)

In [34]:
# Stack image data and create labels
mergers = np.stack(list_of_mergers)
notmergers = np.stack(list_of_nonmergers)
mergers_y = np.ones(mergers.shape[0])
notmergers_y = np.zeros(notmergers.shape[0])

In [35]:
# Combine data and labels
XX = np.vstack((mergers, notmergers))
yy = np.concatenate((mergers_y, notmergers_y))

In [36]:
# Save data and labels
np.save('SB00.npy', XX)
np.save('SB00_y.npy', yy)

In [22]:
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import glob

# Caminhos dos arquivos
target_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\extraidos.hdf5"
txt_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf_snap68.txt"

# Carrega o arquivo de texto com pandas
merged_68 = pd.read_csv(txt_path, sep=" ", header=None, 
                      names=['snap', 'subid', 'logmstar', 'cam', 'merger', 
                             'rfprob', 'asym_I', 'cc_I', 'g_I', 'm20_I', 'mprime_I', 
                             'asym_H', 'cc_H', 'g_H', 'm20_H', 'mprime_H', 
                             't_lastmaj', 't_lastmin', 't_nextmaj', 't_nextmin'])

# Cria um array com objetos usados no RF e aqueles excluídos
rf_objects = np.array([[row['subid'], row['merger']] for _, row in merged_68.iterrows()], dtype=None)

# Obtém IDs de objeto exclusivos e seus rótulos de fusão correspondentes
subid, indices = np.unique(rf_objects[:, 0], return_index=True)
RF_labels = rf_objects[indices]

shid_array = RF_labels[:, 0]
merger_labels = RF_labels[:, 1]

# Define o caminho para o diretório base
base_dir = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\res_subdir"

# Inicializar as listas de imagens
list_of_mergers = []
list_of_nonmergers = []

# Iterar por cada arquivo HDF5 no diretório base
for filename in tqdm(os.listdir(base_dir), desc="Carregando imagens"):
    if filename.endswith(".hdf5"):
        file_path = os.path.join(base_dir, filename)

        # Extrair o ID do objeto a partir do nome do arquivo
        shid = int(filename.split('_')[0])

        # Verificar se o ID do objeto está na lista de IDs
        if shid in shid_array:
            # Obter o índice do ID na lista
            index = np.where(shid_array == shid)[0][0]

            # Obter o rótulo de fusão correspondente
            merger = merger_labels[index]

            # Carregar o arquivo HDF5
            with h5py.File(file_path, "r") as f:
                # Obter a imagem e o rótulo da fusão
                image_f814w = f["f814w"][...]
                image_f160w = f["f160w"][...]

                # Criar um array com as duas imagens
                image_combined = np.stack((image_f814w, image_f160w), axis=0)

                # Adicionar à lista apropriada com base no rótulo de fusão
                if merger:
                    list_of_mergers.append(image_combined)
                else:
                    list_of_nonmergers.append(image_combined)

# Imprime o formato dos dados dentro de cada lista
print("\nFormato dos dados na lista de fusões:")
if list_of_mergers:
    print(np.array(list_of_mergers).shape)
else:
    print("Lista de fusões está vazia.")

print("\nFormato dos dados na lista de não-fusões:")
if list_of_nonmergers:
    print(np.array(list_of_nonmergers).shape)
else:
    print("Lista de não-fusões está vazia.")

# O restante do código para combinar e salvar os dados continua o mesmo...import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import glob

# Caminhos dos arquivos
target_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\extraidos.hdf5"
txt_path = r"C:\Users\stefa\OneDrive\Documentos\Natali 2\Pristine\illustris_morphs_rf_snap68.txt"

# Carrega o arquivo de texto com pandas
merged_68 = pd.read_csv(txt_path, sep=" ", header=None, 
                      names=['snap', 'subid', 'logmstar', 'cam', 'merger', 
                             'rfprob', 'asym_I', 'cc_I', 'g_I', 'm20_I', 'mprime_I', 
                             'asym_H', 'cc_H', 'g_H', 'm20_H', 'mprime_H', 
                             't_lastmaj', 't_lastmin', 't_nextmaj', 't_nextmin'])

# Cria um array com objetos usados no RF e aqueles excluídos
rf_objects = np.array([[row['subid'], row['merger']] for _, row in merged_68.iterrows()], dtype=None)

# Obtém IDs de objeto exclusivos e seus rótulos de fusão correspondentes
subid, indices = np.unique(rf_objects[:, 0], return_index=True)
RF_labels = rf_objects[indices]

shid_array = RF_labels[:, 0]
merger_labels = RF_labels[:, 1]

# Define o caminho para o diretório base
base_dir = r"C:\Users\stefa\OneDrive\Documentos\Natali 2"

# Inicializar as listas de imagens
list_of_mergers = []
list_of_nonmergers = []

# Iterar por cada subdiretório
for subdir in tqdm(os.listdir(base_dir), desc="Carregando imagens"):
    subdir_path = os.path.join(base_dir, subdir)
    if os.path.isdir(subdir_path) and 'res_subdir' in subdir:  # Verifica se o subdiretório contém "res_subdir"
        # Iterar por cada arquivo HDF5 no subdiretório
        for filename in os.listdir(subdir_path):
            if filename.endswith(".hdf5"):
                file_path = os.path.join(subdir_path, filename)
                # Carregar o arquivo HDF5
                with h5py.File(file_path, "r") as f:
                    # Obter a imagem e o rótulo da fusão
                    image_f814w = f["f814w"][...]
                    image_f160w = f["f160w"][...]
                    merger = bool(f["merger"][...])

                    # Criar um array com as duas imagens
                    image_combined = np.stack((image_f814w, image_f160w), axis=0)

                    # Adicionar à lista apropriada com base no rótulo de fusão
                    if merger:
                        list_of_mergers.append(image_combined)
                    else:
                        list_of_nonmergers.append(image_combined)

# Imprime o formato dos dados dentro de cada lista
print("\nFormato dos dados na lista de fusões:")
if list_of_mergers:
    print(np.array(list_of_mergers).shape)
else:
    print("Lista de fusões está vazia.")

print("\nFormato dos dados na lista de não-fusões:")
if list_of_nonmergers:
    print(np.array(list_of_nonmergers).shape)
else:
    print("Lista de não-fusões está vazia.")

# Iterate through objects and their merger labels
for shid, merger in zip(shid_array, merger_labels):
    for camnum in ['00', '01', '02', '03']:
        filter_files = np.sort(np.asarray(glob.glob(os.path.join(base_dir, 'res_sub*', '*sh' + str(shid) + 'cam' + camnum + '*SB00.npy'))))

        # Check if there are exactly 4 files for each object
        if len(filter_files) != 4:
            continue

        # Load image data for all 4 filters
        tmp = np.zeros((4, 75, 75))
        for i, file in enumerate(filter_files):
            tmp[i, :, :] = np.load(file)

        # Append image data to corresponding list based on merger label
        if merger:
            list_of_mergers.append(tmp)
        else:
            list_of_nonmergers.append(tmp)

# Stack image data and create labels
mergers = np.stack(list_of_mergers)
notmergers = np.stack(list_of_nonmergers)
mergers_y = np.ones(mergers.shape[0])
notmergers_y = np.zeros(notmergers.shape[0])

# Combine data and labels
XX = np.vstack((mergers, notmergers))
yy = np.concatenate((mergers_y, notmergers_y))

# Save data and labels
np.save('SB00.npy', XX)
np.save('SB00_y.npy', yy)

# Imprimir formato e tamanho dos arquivos salvos
print(f"\nFormato do arquivo 'SB00.npy': {XX.shape}")
print(f"Tamanho do arquivo 'SB00.npy': {os.path.getsize('SB00.npy')} bytes")
print(f"\nFormato do arquivo 'SB00_y.npy': {yy.shape}")
print(f"Tamanho do arquivo 'SB00_y.npy': {os.path.getsize('SB00_y.npy')} bytes")

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'C:\\Users\\stefa\\OneDrive\\Documentos\\Natali 2\\res_subdir'

In [37]:
import numpy as np

# Load the saved files to get their sizes
XX_loaded = np.load('SB00.npy')
yy_loaded = np.load('SB00_y.npy')

# Print information about the saved arrays
print(f"Data (XX) saved to 'SB00.npy' with shape: {XX_loaded.shape}")
print(f"Labels (yy) saved to 'SB00_y.npy' with shape: {yy_loaded.shape}")

Data (XX) saved to 'SB00.npy' with shape: (16, 2, 75, 75)
Labels (yy) saved to 'SB00_y.npy' with shape: (16,)


In [41]:
# Load the saved files to get their sizes
XX_loaded = np.load('SB00.npy')
yy_loaded = np.load('SB00_y.npy')

In [48]:
xx_data_list = []  # Initialize an empty list to store the selected data
for i in range(len(XX)):  # Loop through all 16 samples (assuming XX is your array)
    xx_data_list.append([XX[i, 0], XX[i, 1]])  # Append a list containing data from channels 0 and 1 for each sample
xx_data = np.stack(xx_data_list)  # Combine the lists into a NumPy array
yy_data = yy  # Assign yy_data to the value of yy

In [49]:
print(XX_loaded.shape) 

(16, 2, 75, 75)


In [51]:
# Generate augmented data by applying transformations
for i in range(len(xx_data)):  # Corrected loop range
    # Up-down flip
    tmp = np.zeros((2, 75, 75))
    tmp[0, :, :] = np.flipud(xx_data[i, 0])
    tmp[1, :, :] = np.flipud(xx_data[i, 1])
    list_ud.append(tmp)

    # Left-right flip
    tmp = np.zeros((2, 75, 75))
    tmp[0, :, :] = np.fliplr(xx_data[i, 0])
    tmp[1, :, :] = np.fliplr(xx_data[i, 1])
    list_lr.append(tmp)

    # 90 degrees rotation
    tmp = np.zeros((2, 75, 75))
    tmp[0, :, :] = np.rot90(xx_data[i, 0])
    tmp[1, :, :] = np.rot90(xx_data[i, 1])
    list_rot.append(tmp)

    # 180 degrees rotation
    tmp = np.zeros((2, 75, 75))
    tmp[0, :, :] = np.rot90(np.rot90(xx_data[i, 0]))
    tmp[1, :, :] = np.rot90(np.rot90(xx_data[i, 1]))
    list_rot180.append(tmp)

In [52]:
# Convert the lists to numpy arrays
mergers_ud = np.stack(list_ud)
mergers_lr = np.stack(list_lr)
mergers_rot = np.stack(list_rot)
mergers_rot180 = np.stack(list_rot180)

In [53]:
# Create labels for the augmented data
y_ud = np.ones(mergers_ud.shape[0])
y_lr = np.ones(mergers_lr.shape[0])
y_rot = np.ones(mergers_rot.shape[0])
y_rot180 = np.ones(mergers_rot180.shape[0])

In [54]:
# Combine the original and augmented data
X_augmented = np.vstack((xx_data[:1624], mergers_ud, mergers_lr, mergers_rot, mergers_rot180, xx_data[1624:]))
y_augmented = np.concatenate((yy_data[:1624], y_ud, y_lr, y_rot, y_rot180, yy_data[1624:]))

In [55]:
# Save the augmented data
np.save('SB00_augmented.npy', X_augmented)
np.save('SB00_augmented_y.npy', y_augmented)

In [56]:
# Print the lengths of the augmented datasets
print(len(X_augmented))
print(len(y_augmented))

144
144
